In [1]:
!pip3 install faiss-gpu faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 25.7 MB/s eta 0:00:0000:0100:01


In [318]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset
from tqdm.auto import tqdm
from fuzzywuzzy import fuzz
from datetime import datetime
import re
import glob
tqdm.pandas()

The plan of action is to use user from [3], book detail from [2], and genre and author from [1].

In [16]:
# os.chdir("/kaggle/input/book-dataset")

# User

In [5]:
users = pd.read_csv("/kaggle/input/book-dataset/3/Users.csv")
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   278858 non-null  int64  
 1   location  278858 non-null  object 
 2   age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [6]:
users.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


# Book Details

## How to Merge Books?

One way is to use ISBNs. The problem with this is that the same book (by the same author) can have different ISBN depending on the publisher. Another approach is to use book title along with the author name. Of course, this will result in some more problems.

The steps are:

1. Using title and author to merge
2. Separate the title, series name and book number
3. Get the author names from author id
4. Do preprocessing to both author name and title
5. Use fuzzy matching with a very high threshold

We will start decoding the author codes. If there are more than one author, we will separate them with a colon (:).

In [7]:
books = load_dataset("parquet", data_files={"train": os.path.join("/kaggle/input/book-dataset/1", "dataset.parquet")})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-8f71964eaf9ea48c/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
books["train"][0:5]

{'authors': ['[1]', '[2, 3]', '[4]', '[5, 6, 7, 8]', '[9]'],
 'categories': ['[214, 220, 237, 2646, 2647, 2659, 2660, 2679]',
  '[235, 3386]',
  '[358, 2630, 360, 2632]',
  '[377, 2978, 2980]',
  '[2813, 2980]'],
 'format': [1.0, 1.0, 1.0, 1.0, 2.0],
 'isbn10': ['184018907X', '184454737X', '8416327866', '571308996', '8352518'],
 'lang': ['en', 'en', 'es', 'en', 'en'],
 'publication-date': ['2004-10-14 00:00:00',
  '2009-03-13 00:00:00',
  '2020-04-30 00:00:00',
  '2015-10-01 00:00:00',
  '2019-06-18 00:00:00'],
 'title': ['Soldier Five : The Real Truth About The Bravo Two Zero Mission',
  'Underbelly : The Gangland War',
  'A Sir Phillip, Con Amor',
  'QI: The Third Book of General Ignorance',
  'The Hidden Power of F*cking Up']}

In [10]:
# books["train"].add_faiss_index(column="format")

## Get Author Name

In [11]:
authors = pd.read_csv('/kaggle/input/book-dataset/1/authors.csv')
authors.head()

,author_id,author_name
0,1,Mike Coburn
1,2,John Silvester
2,3,Andrew Rule
3,4,Julia Quinn
4,5,Andrew Hunter Murray


In [12]:
authors.fillna('Unknown', inplace=True)

In [13]:
# A class is needed if want to run in parallel
# Otherwise will get error like authors is not defined
class AddAuthor:
    def __init__(self, books, authors):
        self.authors = authors
        self.books = books

    def _decode_author(self, row):
        ids = row["authors"]
        ids = eval(ids)
        authors_ = []
        for id_ in ids:
            author = self.authors.iloc[id_ - 1]["author_name"]
            authors_.append(author)
        row["authors_"] = ": ".join(authors_)
        return row
    
    def add_author(self):
        self.books = self.books.map(self._decode_author, batch_size=10000, num_proc=4)
        return self.books

In [14]:
#the same function that is used in the class above
def _decode_author(row):
    ids = row["authors"]
    ids = eval(ids)
    authors_ = []
    for id_ in ids:
        author = authors.iloc[id_ - 1]["author_name"]
        authors_.append(author)
    row["authors_"] = ": ".join(authors_)
    return row

In [15]:
add_author = AddAuthor(books, authors)
books = add_author.add_author()

#0:   0%|          | 0/277346 [00:00<?, ?ex/s]

#1:   0%|          | 0/277346 [00:00<?, ?ex/s]

#2:   0%|          | 0/277346 [00:00<?, ?ex/s]

#3:   0%|          | 0/277345 [00:00<?, ?ex/s]

In [16]:
books["train"][:5]

{'authors': ['[1]', '[2, 3]', '[4]', '[5, 6, 7, 8]', '[9]'],
 'categories': ['[214, 220, 237, 2646, 2647, 2659, 2660, 2679]',
  '[235, 3386]',
  '[358, 2630, 360, 2632]',
  '[377, 2978, 2980]',
  '[2813, 2980]'],
 'format': [1.0, 1.0, 1.0, 1.0, 2.0],
 'isbn10': ['184018907X', '184454737X', '8416327866', '571308996', '8352518'],
 'lang': ['en', 'en', 'es', 'en', 'en'],
 'publication-date': ['2004-10-14 00:00:00',
  '2009-03-13 00:00:00',
  '2020-04-30 00:00:00',
  '2015-10-01 00:00:00',
  '2019-06-18 00:00:00'],
 'title': ['Soldier Five : The Real Truth About The Bravo Two Zero Mission',
  'Underbelly : The Gangland War',
  'A Sir Phillip, Con Amor',
  'QI: The Third Book of General Ignorance',
  'The Hidden Power of F*cking Up'],
 'authors_': ['Mike Coburn',
  'John Silvester: Andrew Rule',
  'Julia Quinn',
  'Andrew Hunter Murray: James Harkin: John Lloyd: John Mitchinson',
  'The Try Guys']}

We have authors now. Let's load the other book data from [2] and have a look at the first 5 items:

In [163]:
books2 = load_dataset("csv", data_files="/kaggle/input/book-dataset/2/book1-100k.csv", delimiter=",")
books2["train"][:5]

  0%|          | 0/1 [00:00<?, ?it/s]

{'id': [1, 2, 3, 4, 5],
 'title': ['Harry Potter and the Half-Blood Prince (Harry Potter, #6)',
  'Harry Potter and the Order of the Phoenix (Harry Potter, #5)',
  "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",
  'Harry Potter and the Chamber of Secrets (Harry Potter, #2)',
  'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)'],
 'authors': ['J.K. Rowling',
  'J.K. Rowling',
  'J.K. Rowling',
  'J.K. Rowling',
  'J.K. Rowling'],
 'pages': [652, 870, 309, 352, 435],
 'language': ['eng', 'eng', 'eng', 'eng', 'eng'],
 'star_1': [9896, 12455, 108202, 11896, 10128],
 'star_2': [25317, 37005, 130310, 49353, 24849],
 'star_3': [159960, 211781, 567458, 288821, 194848],
 'star_4': [556485, 604283, 1513191, 706082, 630534],
 'star_5': [1546466, 1493113, 4268227, 1504505, 1749958],
 'rating': [4.57, 4.5, 4.47, 4.42, 4.57],
 'total_ratings': [2298124, 2358637, 6587388, 2560657, 2610317],
 'total_reviews': [28062, 29770, 75911, 244, 37093],
 'isbn': [None, '0439358078', None,

## Get Book Name and Series Name

First five books are from Harry Potter series. Let's see what detail is available for these books in the dataset [1]:

In [164]:
JK = books["train"].filter(lambda row: "J.K. Rowling" in row["authors_"])
JK[:5]

  0%|          | 0/1110 [00:00<?, ?ba/s]

{'authors': ['[191489, 191490]',
  '[191489]',
  '[191489]',
  '[191489]',
  '[191489]'],
 'categories': ['[218, 222, 292, 821, 3366, 2804, 2812, 2813]',
  '[355, 2629, 2496]',
  '[355, 2629, 2496]',
  '[355, 2629, 2496]',
  '[355, 2629, 2496]'],
 'format': [2.0, 3.0, 3.0, 3.0, 3.0],
 'isbn10': ['1408706784',
  '1408824094',
  '1408824132',
  '1408821583',
  '1408821516'],
 'lang': ['en', 'en', 'en', 'en', 'en'],
 'publication-date': ['2015-04-14 00:00:00',
  '2011-04-04 00:00:00',
  '2011-05-03 00:00:00',
  '2011-05-03 00:00:00',
  '2011-05-19 00:00:00'],
 'title': ['Very Good Lives : The Fringe Benefits of Failure and the Importance of Imagination',
  'Harry Potter and the Chamber of Secrets',
  'Harry Potter and the Prisoner of Azkaban',
  'Harry Potter and the Goblet of Fire',
  'Harry Potter and the Chamber of Secrets'],
 'authors_': ['J.K. Rowling: Joel Holland',
  'J.K. Rowling',
  'J.K. Rowling',
  'J.K. Rowling',
  'J.K. Rowling'],
 'ID': [235830, 265296, 265297, 265300, 26530

We can see that the books title does not have the series name in [1] as it is in [2]. This means that we can not directly use the title to merge the two dataframes. We will have to extract the book name first. This is what we will do next. Fortunately, the book name, the series name and the book number are separated in a logical way and we can use RegEx to extract them.

In [165]:
# Again, we have to use a class if you don't want to import re every time you call the function:
class GetBookInfo:
    import re
    def __init__(self, books):
        self.books = books
        self.regex = re.compile(r"(.*?)\s\((.*?),?\s#(\d{1,3}-?\d{0,3})\)")

    def get_book_info(self, row):
        title = row["title"]
        matches = self.regex.findall(title)
        if len(matches) == 0:
            row["title_"] = title
            row["series"] = "Standalone"
            row["book_num"] = 1
            return row 
        matches = matches[0]
        series = matches[1]
        title_ = matches[0]
        book_num = matches[2]
        if "-" in book_num:
            book_num = int(book_num.split("-")[-1])
        row["title_"] = title_
        row["series"] = series
        row["book_num"] = int(book_num)
        return row
    
    def map_book_info(self):
        self.books = self.books.map(self.get_book_info, batch_size=5000, num_proc=4)
        return self.books

In [166]:
# Here is the same function used in above class
def get_series(row):
    import re
    title = row["title"]
    regex = re.compile(r"(.*?)\s\((.*?),?\s#(\d{1,3}-?\d{0,3})\)")
    matches = regex.findall(title)
    if len(matches) == 0:
        row["title_"] = title
        row["series"] = "Standalone"
        row["book_num"] = 1
        return row 
    matches = matches[0]
    series = matches[1]
    title_ = matches[0]
    book_num = matches[2]
    if "-" in book_num:
        book_num = int(book_num.split("-")[-1])
    row["title_"] = title_
    row["series"] = series
    row["book_num"] = int(book_num)
    return row

In [167]:
book_info_getter = GetBookInfo(books2)
books2 = book_info_getter.map_book_info()

#0:   0%|          | 0/14573 [00:00<?, ?ex/s]

#1:   0%|          | 0/14573 [00:00<?, ?ex/s]

#2:   0%|          | 0/14573 [00:00<?, ?ex/s]

#3:   0%|          | 0/14573 [00:00<?, ?ex/s]

We will have to run this for all the book dataframes in [2]. Let's see the output:

In [168]:
books2["train"][:5]

{'id': [1, 2, 3, 4, 5],
 'title': ['Harry Potter and the Half-Blood Prince (Harry Potter, #6)',
  'Harry Potter and the Order of the Phoenix (Harry Potter, #5)',
  "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",
  'Harry Potter and the Chamber of Secrets (Harry Potter, #2)',
  'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)'],
 'authors': ['J.K. Rowling',
  'J.K. Rowling',
  'J.K. Rowling',
  'J.K. Rowling',
  'J.K. Rowling'],
 'pages': [652, 870, 309, 352, 435],
 'language': ['eng', 'eng', 'eng', 'eng', 'eng'],
 'star_1': [9896, 12455, 108202, 11896, 10128],
 'star_2': [25317, 37005, 130310, 49353, 24849],
 'star_3': [159960, 211781, 567458, 288821, 194848],
 'star_4': [556485, 604283, 1513191, 706082, 630534],
 'star_5': [1546466, 1493113, 4268227, 1504505, 1749958],
 'rating': [4.57, 4.5, 4.47, 4.42, 4.57],
 'total_ratings': [2298124, 2358637, 6587388, 2560657, 2610317],
 'total_reviews': [28062, 29770, 75911, 244, 37093],
 'isbn': [None, '0439358078', None,

It is working as intended. Now, the next step is to use these to merge the dataframes.

## Merging the Dataframes

To merge, we will check similarity between the title+author. We will be using fuzzy matching for now. Later, we might try using embeddings. Here is the algorithm:

- Clean the author name and title by removing any special characters and converting to lower case.
- Give different weight to author matching and the title matching, say 0.7 and 0.3 respectively.
<!-- - Use `partial_ratio` from `fuzzywuzzy` to get the similarity score for author as well as title. Set a threshold that must be met for the two books to be considered same. Say 90%. -->
<!-- - The function must return two rows: the ID of the book matched and the score. The map function will run on the smaller dataframe [2]. -->
- We will be using NLP models.

### Setting Up

In [259]:
from transformers import AutoTokenizer, AutoModel
import torch

model_ckpt = "sentence-transformers/paraphrase-MiniLM-L3-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

device = torch.device("cuda")
model = model.to(device)
model = model.eval()

In [171]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return model_output.last_hidden_state[:, 0]

In [175]:
ids = np.arange(0,len(books["train"]),1)
books["train"] = books["train"].add_column("ID", ids)

In [176]:
books

DatasetDict({
    train: Dataset({
        features: ['authors', 'categories', 'format', 'isbn10', 'lang', 'publication-date', 'title', 'authors_', 'ID'],
        num_rows: 1109383
    })
})

In [286]:
books_ = books.remove_columns(["text", "text_embeddings"])
books_["train"].to_parquet("all_books_with_id.parquet")

204918380

We will create a new column, `text` that is a combination of `{title} by {authors}`:

In [178]:
def create_text(row):
    title = row['title']
    authors = row['authors_']
    authors = authors.replace(":", " and")
    row["text"] =  f"{title} by {authors}"
    return row

In [179]:
books = books.map(create_text, batch_size = 5000,
    num_proc = 2)

  0%|          | 0/1109383 [00:00<?, ?ex/s]

In [184]:
books = books.map(
    lambda x: {"text_embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]},
    batch_size = 5000,
#     num_proc = 2
)


  0%|          | 0/1109383 [00:00<?, ?ex/s]

In [186]:
books["train"].add_faiss_index(column="text_embeddings")
books

  0%|          | 0/1110 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['authors', 'categories', 'format', 'isbn10', 'lang', 'publication-date', 'title', 'authors_', 'ID', 'text', 'text_embeddings'],
        num_rows: 1109383
    })
})

In [289]:
books = books["train"]

The following class, with some modifications, can be used in windows. However, if on kaggle, just use the function below:

### The Main Code to Merge Books

In [ ]:
threshold = 10 # The minimum FAISS score to be considered a match

In [157]:
# A class is needed if want to run in parallel in Windows
class MatchBook:
    def __init__(self, all_books, books):
        self.all_books = all_books
        self.books = books
        self.model_ckpt = "sentence-transformers/paraphrase-MiniLM-L3-v2"
        self.tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        model = AutoModel.from_pretrained(model_ckpt)
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.model = model.to(device)
    
    def cls_pooling(self, model_output):
        return model_output.last_hidden_state[:, 0]
    
    def change_books(self, new_books):
        self.books = new_books
    
    def get_embeddings(self, text_list):
        encoded_input = self.tokenizer(
            text_list, padding=True, truncation=True, return_tensors="pt"
        )
        encoded_input = {k: v.to(self.device) for k, v in encoded_input.items()}
        model_output = self.model(**encoded_input)
        return self.cls_pooling(model_output)

    def _match_book(self, row):
        title = row["title_"]
        author = row["authors"]
        text = f"{title} by {author}"
        text_embedding = self.get_embeddings([text]).cpu().detach().numpy()
        scores, samples = self.all_books.get_nearest_examples(
            "text_embeddings", text_embedding, k=1
        )
        if scores[0]<=threshold:
            row["book_1_id"] = samples["ID"][0]
            row["score"] = scores[0]
        else:
            row["book_1_id"] = -1
            row["score"] = scores[0]
        return row

    
    def match_book(self):
        self.books_ = self.books.map(self._match_book, batch_size=2000, num_proc=1)
        return self.books_

In [290]:
def match_book_batched(rows):
    """Used when in batched and want to return tuple"""
    titles = rows["title_"]
    authors = rows["authors"]
    text = [f"{title} by {author}" for title, author in zip(titles, authors)]
    text_embeddings = get_embeddings(text).cpu().detach().numpy()
    scores, samples = books.get_nearest_examples_batch(
        "text_embeddings", text_embeddings, k=1
    )
    scores = np.array(scores)
    scores = np.squeeze(scores)
    mask = scores<=threshold
    book_ids = np.ones_like(mask)*-1
    sample_ids = np.array([sample["ID"][0] for sample in samples])
    book_ids[mask] = sample_ids[mask]
    return book_ids, scores


In [291]:
def match_book(row):
    """Use when a single item is passed to the function"""
    title = row["title_"]
    author = row["authors"]
    text = f"{title} by {author}"
    text_embedding = get_embeddings([text]).cpu().detach().numpy()
    scores, samples = books.get_nearest_examples(
        "text_embeddings", text_embedding, k=1
    )
    if scores[0]<=threshold:
        book_ids= samples["ID"][0]
        scores = scores[0]
    else:
        book_ids = -1
        scores = scores[0]
    row["book_id"] = book_ids
    row["score"] = scores
    return row


In [295]:
def match_book_batched_row(rows):
    """Use when batch is passed along with the map function"""
    titles = rows["title_"]
    authors = rows["authors"]
    text = [f"{title} by {author}" for title, author in zip(titles, authors)]
    text_embeddings = get_embeddings(text).cpu().detach().numpy()
    scores, samples = books.get_nearest_examples_batch(
        "text_embeddings", text_embeddings, k=1
    )
    scores = np.array(scores)
    scores = np.squeeze(scores)
    mask = scores<=threshold
    book_ids = np.ones_like(mask)*-1
    sample_ids = np.array([sample["ID"][0] for sample in samples])
    book_ids[mask] = sample_ids[mask]
    rows["book_id"] = book_ids
    rows["score"] = scores
    return rows


In [293]:
books2 = books2["train"]

In [296]:
books2_ = books2.map(match_book_batched_row, batch_size = 800, batched=True)

  0%|          | 0/73 [00:00<?, ?ba/s]

In [ ]:
books2_.to_parquet("book1-100k.parquet")

In [301]:
torch.cuda.empty_cache()

In [323]:
all_books = glob.glob("/kaggle/input/book-dataset/2/*.csv")
all_books.sort()
len(all_books)

23

In [332]:
def merge_one(file_dir):
    """Merges one smaller book file with the larger book file."""
    print(f"Working on: {file_dir}")
    books2 = load_dataset("csv", data_files=file_dir, delimiter=",")
    book_info_getter = GetBookInfo(books2)
    books2 = book_info_getter.map_book_info()
    books2 = books2.map(match_book_batched_row, batch_size = 800, batched=True)
    file_name = file_dir.split(os.path.sep)[-1]
    file_name = file_name.replace("csv", "parquet")
    books2_.to_parquet(file_name)
    torch.cuda.empty_cache()
    print(f"saved as: {file_name}")

In [ ]:
merge_one(all_books[2])

Working on: /kaggle/input/book-dataset/2/book100k-200k.csv


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-774d795c4601ef0b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

#1:   0%|          | 0/14262 [00:00<?, ?ex/s]

#0:   0%|          | 0/14262 [00:00<?, ?ex/s]

#2:   0%|          | 0/14261 [00:00<?, ?ex/s]

#3:   0%|          | 0/14261 [00:00<?, ?ex/s]

  0%|          | 0/72 [00:00<?, ?ba/s]

In [275]:
# # %%timeit
# start_id = 20
# items = 1000
# book_ids_b, scores_b = match_book_batched(books2["train"][start_id:start_id+items])

In [277]:
# def get_batch(dataset, batch_size = 320):
#     start = 0
#     end = len(dataset)
#     for i in range(start, end, batch_size):
#         if (i+batch_size)>end:
#             j = end
#         else:
#             j = i +batch_size
#         yield dataset[i:j]

In [276]:
# batch_size = 800
# batched_ds = get_batch(books2["train"], batch_size=batch_size)
# book_ids_all = []
# score_all = []
# for b in tqdm(batched_ds, desc="Getting ids",total = len(books2["train"])//batch_size):
#     book_ids_b, scores_b = match_book_batched(b)
#     book_ids_all.extend(book_ids_b)
#     score_all.extend(scores_b)

In [123]:
# %time
# start_id = 20
# items = 100
# book_ids, scores = match_book(books2["train"][start_id:start_id+items])

In [127]:
# %%timeit
# start_id = 20
# items = 1000
# book_ids = []
# scores = []
# for i in range(start_id, start_id+items):
#     book_id, score = match_book(books2["train"][i:i+1])
#     book_ids.append(book_id)
#     scores.append(score)

7.69 s ± 126 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [298]:
# start_id = 20
# items = 10
# titles = books2["train"][start_id:start_id+items]["title_"]
# authors = books2["train"][start_id:start_id+items]["authors"]

# texts = [f"{title} by {author}" for title, author in zip(titles, authors)]
# for i, text in enumerate(texts):
#     print(text)
#     text_embedding = get_embeddings([text]).cpu().detach().numpy()
#     scores, samples = filtered_books.get_nearest_examples_batch(
#         "text_embeddings", text_embedding, k=1
#     )
#     if scores[0]<=threshold:
#         print("ID: ", samples["ID"][0])
#         print("Score: ", scores[0])
#         print("Match: ", samples["text"][0])
#     else:
#         print("No Match")
#     print("----"*10)

### Using non-NLP Method

The class below can be used to merge books using the title and author name. However, it is very slow and hence is not used.

In [25]:
class BookMatching:
    import re
    from fuzzywuzzy import fuzz
    def __init__(self, all_books, books):
        """all_books: list of all books in the database
           books: list of books to be matched"""
        self.all_books = all_books
        self.books = books
        self.books_all_titles = self.all_books[:]["title_"]
        self.books_all_authors = self.all_books[:]["authors"]
        self.author_weight = 0.7 # give more weight to author name
        self.title_weight = 0.3
        self.threshold = 90 # 90% match
        self.final_results = []
        self.regex = re.compile('[^a-zA-Z]')
        self.fuzz_function = fuzz.token_set_ratio

    def clean_name(self, name):
        name = self.regex.sub('', name)
        return name.lower()
    
    def calculate_score(self, author1, author2, title1, title2):
        author1 = self.clean_name(author1)
        author2 = self.clean_name(author2)
        title1 = self.clean_name(title1)
        title2 = self.clean_name(title2)
        author_score = 0
        title_score = 0
        if author1 == author2:
            author_score = 100
        else:
            author_score = self.fuzz_function(author1, author2)
        if title1 == title2:
            title_score = 100
        else:
            title_score = self.fuzz_function(title1, title2)
        return author_score*self.author_weight + title_score*self.title_weight

    def map_function(self, row):
        title = row["title"]
        author = row["authors_"]
        max_score = 0
        max_idx = 0
        for i in range(len(self.books_all_titles)):
            score = self.calculate_score(author, self.books_all_authors[i], title, self.books_all_titles[i])
            if score > max_score:
                max_score = score
                max_idx = i
            if max_score == 100:
                break
        if not max_score >= self.threshold:
            row["book_id"] = -1
            row["score"] = -1
        else:
            row["book_id"] = max_idx
            row["score"] = max_score
        return row

    def match(self):
        self.books_u = self.books.map(self.map_function, batch_size=10, num_proc=8)
        return self.books_u

In [299]:
# filtered_books = books["train"].filter(lambda row: "355" in row["categories"])
# filtered_books

In [300]:
# book_matcher = BookMatching(books2["train"], filtered_books)
# books2_ = book_matcher.match()